In [ ]:
import os
import pandas as pd
from torchvision.io import read_image


In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [2]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
optimizer = Adam(learning_rate=.002)
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def load_train(path):
    datagen = ImageDataGenerator(validation_split=0.25, rescale=1./255)
    train_datagen_flow = train_datagen.flow_from_dataframe(dataframe=pd.read_csv(path+'labels.csv'),
    directory='/datasets/faces/final_files/',
    x_col='file_name',
    y_col='real_age',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    seed=2807, 
    subset='training') 
    
    return train_datagen_flow 

In [ ]:
def load_test(path):
    datagen = ImageDataGenerator(validation_split=0.25, rescale=1./255)
    test_datagen_flow = train_datagen.flow_from_dataframe(dataframe=pd.read_csv(path+'labels.csv'),
    directory='/datasets/faces/final_files/',
    x_col='file_name',
    y_col='real_age',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    seed=2807, 
    subset='validation') 
    
    return test_datagen_flow

In [8]:
def create_model(input_shape):    
    model = Sequential()
    model.add(Conv2D(filters=6,
                 kernel_size=(5, 5),
                 padding='same',
                 activation='relu',
                 input_shape=(input_shape)))
    model.add(AvgPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units=1, activation='relu'))
    model.compile(loss='mean_squared_error', 
              optimizer='adam', metrics=[tensorflow.keras.metrics.MeanAbsoluteError()])
    return model 

In [4]:
def train_model(model, train_data, test_data, batch_size=None, epochs=5,
                steps_per_epoch=None, validation_steps=None):
    model.fit(train_data,
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2)
    return model